# Entity Matching (EM) about Books

# Introduction

This IPython notebook shows a basic workflow two tables using *py_entitymatching*. We want to match data science books in library of UW-Madison and UIUC.  The book information of UW-Madison is from [here](https://search.library.wisc.edu/search/system?q=Data+Science) and the book information of UIUC is from [here](https://vufind.carli.illinois.edu/vf-uiu/Search/Home?lookfor=Data+Science+&type=all&start_over=1&submit=Find&search=new). Details can be found from our Stage 2 Report [here](https://github.com/iphyer/CS839ClassProject/blob/master/stage2/Stage2Report.pdf). 


First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import pandas as pd
import py_entitymatching as em

# Read input tables

We begin by loading the input tables.

We name the table about UW-Madison `TableA.csv` and the table about UIUC `TableB.csv`. And there are 

* 6963 tuples in table `TableA.csv`
* 5730 tuples in table `TableB.csv`

In [2]:
A = em.read_csv_metadata('../data/TableA.csv', key = 'ID')
B = em.read_csv_metadata('../data/TableB.csv', key = 'ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [3]:
# block_f = em.get_features_for_blocking(A, B)

# Block tables to get candidate set

Here we will use several blockers to remove obviously non-matching tuple pairs from the input tables.

For the same book, since we got the data from two different library websites, their attributes may not be the exact same. Therefore, we applied an OverlapBlocker over some of the attributes, including the *Title*, *Author* and *Series* of the book.

After multiple tests, we found the best overlap_size for each attribute - for *Title*, *Author* and *Series*, we set the overlap_size to be 1, 3 and 1 respectively.

In [4]:
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'Author', 'Author', l_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], r_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], overlap_size = 1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [5]:
D = ob.block_candset(C, 'Title', 'Title', overlap_size = 3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


In [6]:
E = ob.block_candset(D, 'Series', 'Series', overlap_size = 1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [7]:
F = ob.block_candset(E, 'Publication', 'Publication', overlap_size = 1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
# G = em.label_table(F, label_column_name='gold_labels')

In [ ]:
S = em.sample_table(F, 500)

In [ ]:
S

In [ ]:
# block_f = em.get_features_for_blocking(A, B)

# CODE 

In [ ]:
S[S['ltable_ISBN'] == S['rtable_ISBN']].shape

In [ ]:
len(F)

In [ ]:
s1 = pd.merge(A, B, how='inner', on=['ISBN'])

In [ ]:
s1.head(20)

In [ ]:
C[C['ltable_ISBN'] == C['rtable_ISBN']].shape

In [ ]:
D[D['ltable_ISBN'] == D['rtable_ISBN']].shape

In [ ]:
E[E['ltable_ISBN'] == E['rtable_ISBN']].shape

In [ ]:
F[F['ltable_ISBN'] == F['rtable_ISBN']].shape

In [ ]:
D = ob.block_candset(block_data, 'Title', 'Title', allow_missing=True)

In [ ]:
em.get_key(data1)

In [ ]:
data1.keys()